In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import re
import string
import time

### 1 определяем две функции:
1. car_write_to_file(url, filename, car_id) - парсит одно объявление и добавляет данные в выходной файл
2.  urls_write_to_file(FILENAME_URL) - создает перечень ссылок на объявления для конкретной марки,
указанной в глобальной переменной brand

In [ ]:
## Функция по парсингу одного объявления по известному URL в файл

def car_write_to_file(url, filename, car_id):
    global feature_name_kaggle, keys_list_1, keys_list_2, keys_2_site, feature_name_dict2
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    the_car_features = dict.fromkeys(feature_name_kaggle, '')
    the_car_features['id'] = car_id
    the_car_features['URL'] = url
    try:
        the_car_features['name'] = soup.select(".CardBreadcrumbs__itemText")[5].text[:-1]
    except:
        pass
    
    # набор признаков №1 (список 1)
    for key in keys_list_1:
        try:
            the_car_features[key] = soup.find(itemprop = key).attrs['content']
        except:
            print(' НЕ НАЙДЕНО --- ', key, url)

    
    
    # набор признаков №2 (список 2)
    for li in soup.select(".CardInfo > li"):
        feature_name_span, value_span = li.find_all('span')
        try:
            key = feature_name_dict2[feature_name_span.text]  # как на каггле называется этот параметр
            the_car_features[key] = value_span.text
        except:
            pass

    try:
        txt = the_car_features['description']
        the_car_features['description'] = '"' + re.sub(r'[^A-zА-я0-9 ]', ' ', txt) + '"'
    except:
        pass
    
    # в километраже оставить только цифры
    try:
        the_car_features['mileage'] = re.sub(r'[^0-9]', '', the_car_features['mileage'])
    except:
        pass


    # Цена - оставить только цифры
    try:
        the_car_features['Price_RU'] = re.sub(r'[^0-9]', '', soup.select(".OfferPriceCaption__price")[0].text)
    except:
        pass
    
    # Комплектация
    try:
        complectation_search = soup.find("script", id="initial-state").text
        start = complectation_search.find('"equipmentGroups":')
        end = complectation_search.rfind('},"lk_summary"')
        if start != -1:
            the_car_features['Комплектация'] = "['" + complectation_search[start + len('"equipmentGroups":'):end] + "']"
        else:
            the_car_features['Комплектация'] = []
    except:
        pass
    
  

    text_list = []
    for key in feature_name_kaggle:
        text_list.append(the_car_features[key])
    text = "|".join(map(str, text_list))
    with open (FILENAME, 'a') as file:
        file.write(text + '\n')


# Функция по созданию списка ссылок на объявления
# проходит по регионам из списка cities_list, для каждого региона выбираются поочередно года с 1970 по 2019 
# (цикл for year in range(1970, 2020))
# и берет ссылки на конкретные объявления по этим страницам

def urls_write_to_file(FILENAME_URL):
    global brand
    PARAMS_start = {'page' : '1',
              'output_type' : 'list',
              'sort' : 'fresh_relevance_1-desc'}

    offer_count = 0
    cities_list = ['adygeya', 'altay', 'bashkortostan', 'buryatiya', 'dagestan', 'ingushetiya', 'kabardino-balkariya', 'kalmykiya', 'karachaevo-cherkesiya',
        'kareliya', 'komi', 'krym', 'mariy_el', 'mordoviya', 'saha_yakutiya', 'severnaya_osetiya', 'tatarstan', 'tyva',
        'udmurtiya', 'hakasiya', 'chechenskaya_respublika', 'chuvashskiya', 'altayskiy_kray', 'zabaykalskiy_kray', 'kamchatskiy_kray',
        'krasnodarskiy_kray', 'krasnoyarskiy_kray', 'permskiy_kray', 'primorskiy_kray', 'stavropolskiy_kray', 'habarovskiy_kray',
        'amurskaya_oblast', 'arhangelskaya_oblast', 'astrahanskaya_oblast', 'belgorodskaya_oblast', 'bryanskaya_oblast', 'vladimirskaya_oblast',
        'volgogradskaya_oblast', 'vologodskaya_oblast', 'voronezhskaya_oblast', 'ivanovskaya_oblast', 'irkutskaya_oblast',
        'kaliningradskaya_oblast', 'kaluzhskaya_oblast', 'kemerovo', 'kirovskaya_oblast', 'kostromskaya_oblast', 'kurganskaya_oblast', 
        'kurskaya_oblast', 'leningradskaya_oblast', 'lipetskaya_oblast', 'magadanskaya_oblast', 'moskovskaya_oblast', 'murmanskaya_oblast',
        'nizhegorodskaya_oblast', 'novgorodskaya_oblast', 'novosibirskaya_oblast', 'omskaya_oblast', 'orenburgskaya_oblast', 'orlovskaya_oblast',
        'penzenskaya_oblast', 'pskovskaya_oblast', 'rostovskaya_oblast', 'ryazanskaya_oblast', 'samarskaya_oblast', 'saratovskaya_oblast',
        'sahalinskaya_oblast', 'sverdlovskaya_oblast', 'smolenskaya_oblast', 'tambovskaya_oblast', 'tverskaya_oblast', 'tomskaya_oblast',
        'tulskaya_oblast', 'tyumenskaya_oblast', 'ulyanovskaya_oblast', 'chelyabinskaya_oblast', 'yaroslavskaya_oblast',
        'moskva', 'sankt-peterburg', 'sevastopol', 'nenetskiy_ao', 'hanty-mansiyskiy_ao', 'chukotskiy_ao', 'yamalo-nenetskiy_ao',
        'vladimir', 'ryazan', 'nizhniy_novgorod', 'murom', 'tver', 'dubna', 'kolomna', 'tula', 'kaluga', 'vyazma', 'rzhev', 'smolensk',
        'roslavl', 'bryansk', 'pskov', 'elets', 'lipetsk', 'tambov', 'penza', 'saransk', 'ulyanovsk', 'tolyatti', 'samara', 'kazan',
        'yoshkar-ola', 'cheboksary', 'kostroma', 'yaroslavl', 'ivanovo', 'cherepovets', 'vologda', 'kirovskaya_oblast_kirov', 'perm',
        'ekaterinburg', 'chelyabinsk', 'tyumen', 'ufa', 'krasnoyarsk', 'irkutsk']    # 'kaliningradskaya_oblast', 
    for city in cities_list:
        print('city: ', city)
        for year in range(1970, 2020):
            URL = 'https://auto.ru/' + city + '/cars/' + str(brand) + '/' + str(year) + '-year/used/'
            response = requests.get(URL, params = PARAMS_start)
#            print (response.url)
            soup = BeautifulSoup(response.content)
            try:
                offer_number = int(re.sub(r'[^0-9]', '', soup.select(".ButtonWithLoader__content")[0].text))
            except:
                offer_number = 0
            offer_count += offer_number
            page_number = offer_number // 37 + 1
#            print('Total offers: {}, pages: {}'.format(offer_number, page_number))
            for page in range (page_number):
                PARAMS = {'page' : page,
                          'output_type' : 'list',
                          'sort' : 'fresh_relevance_1-desc'}
                response = requests.get(URL, params = PARAMS)
                soup = BeautifulSoup(response.content)
                for elem in soup.find_all("meta", itemprop = "url"):
                    url_cur = elem.attrs['content']
                    with open (FILENAME_URL, 'a') as file:
                        file.write(city + ',' + url_cur + '\n')
    print('TOTAL OFFERS: ', offer_count)


### 2 Задаем глобальные переменные - список интересующих признаков и наименование бренда автомобиля

In [ ]:
# определяем наименование полей для нашего датасета

feature_name_kaggle = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
                       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
                       'vehicleTransmission', 'engineDisplacement', 'enginePower',
                       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
                       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id',
                      'price', 'URL']
# определяем для удобства две группы полей, которые похожим образом парсятся на странце
keys_list_1 =  ['bodyType', 'brand', 'color', 'fuelType', 'modelDate',
              'numberOfDoors', 'productionDate', 'vehicleConfiguration',
             'vehicleTransmission', 'engineDisplacement', 'enginePower',
             'description', 'price']
keys_list_2 = ['mileage', 'Привод', 'Руль', 'Состояние',
            'Владельцы', 'ПТС', 'Таможня', 'Владение']
keys_2_site = ['Пробег', 'Привод', 'Руль', 'Состояние', 
               'Владельцы', 'ПТС', 'Таможня', 'Владение']

feature_name_dict2 = dict(zip(keys_2_site, keys_list_2))

# задаем наименование бренда, который ищем в глобальной переменной brand

brand = 'toyota'

FILENAME = brand + '_train_data.csv'
FILENAME_URL = brand + '_urls_list.csv'
FILENAME_URL_unique = brand + '_URLs_unique.csv'
car_id_continue = 0
FILENAME

### 3 Основной скрипт:
сначала считываем в файл все ссылки на интересующие объявления,

потом идем по этому файлу и заполняем датасет

In [ ]:
with open (FILENAME_URL, 'w') as file:
    file.write('city,offer_URL' + '\n')

# считываем в файл все ссылки на интересующие объявления,

urls_write_to_file(FILENAME_URL)

# прочитаем получившийся файл и удалим дубликаты:

df_urls = pd.read_csv(FILENAME_URL)
print('С дубликатами: ', df_urls.shape)
df_urls = df_urls.drop_duplicates(subset = 'offer_URL')
print('Осталось: ', df_urls.shape)

df_urls.to_csv(FILENAME_URL_unique)

# идем по этому файлу и заполняем датасет
with open (FILENAME, 'w') as file:
    file.write("|".join(map(str, feature_name_kaggle)) + '\n')

# Если следующий цикл отработает с ошибкой и запишется только часть объявления, 
#то можно считать заново список уникальных URL и дополнять файл, 
# начиная с конкретного car_id_continue: df_urls = pd.read_csv(FILENAME_URL_unique)

car_id = car_id_continue
total_urls = df_urls.shape[0]
while car_id < total_urls:
    try:    
        for url_cur in df_urls.iloc[car_id_continue:]['offer_URL']:
            car_write_to_file(url_cur, FILENAME, car_id)
            car_id += 1
            time.sleep(1)
            if car_id % 100 == 0: print('car_id: ', car_id)
    except:
        # если что-то пойдет не так, например разоврется соединение, то ждем 5 мин и пробуем продолжать
        time.sleep(300)
        my_data = pd.read_csv(FILENAME, encoding = 'ANSI', sep ='|')
        car_id_continue = my_data.shape[0]
        print('======  except block ==============')

print('----end---')

### Если цикл по парсингу списка уникальных адресов (объявлений) завершился аварийно,
то можно прочитать тот датасет, что смогли записать и продолжать с последнего объявления- вернутья в п.3

In [ ]:
my_data = pd.read_csv(FILENAME, encoding = 'ANSI', sep ='|')
car_id_continue = my_data.shape[0]
car_id_continue